In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
matplotlib.rcParams['figure.figsize'] = (20, 10)

In [ ]:
# Если хочется, ниже можно будет проинферить фотки

# Для этого сначала загрузим модель

from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict
from ultralytics.engine.results import Results

model = AutoDetectionModel.from_pretrained(
            model_type='yolov10',
            model_path='/home/slava/LCT_24/random/sasha/10s_1280.pt',
            confidence_threshold=0.25,
            device='cuda')

# И ещё нужна будет вспомогательная функция

def get_yolo_result(result_sahi, category_mapping):
    if len(result_sahi.object_prediction_list) > 0:
        bboxes = torch.stack(
            [
                torch.Tensor(object_prediction.bbox.to_xyxy()) for object_prediction in result_sahi.object_prediction_list]
        )
        scores = torch.Tensor(
            [object_prediction.score.value for object_prediction in result_sahi.object_prediction_list]
        )
        categories = torch.Tensor(
            [object_prediction.category.id for object_prediction in result_sahi.object_prediction_list]
        )
        try:
            result_tensor = torch.cat([
                bboxes,
                scores.unsqueeze(0).T,
                categories.unsqueeze(0).T], axis=1)
        except:
            import ipdb; ipdb.set_trace()
    
    else:
        result_tensor = None

    yolo_result = Results(
        np.array(result_sahi.image),
        path=None,
        names=category_mapping,
        boxes=result_tensor)
    return yolo_result

def convert_category_keys(category_mapping):
    return dict(
                zip(
                    list(map(int, category_mapping.keys())),
                    category_mapping.values(),
                )
            )

In [ ]:
# Словарь для меток
labels = {
    0: 'copter_type_uav',
    1: 'aircraft',
    2: 'helicopter',
    3: 'bird',
    4: 'aircraft_type_uav'
}

# Загрузка данных из CSV файла
file_path = '/home/slava/LCT_24/exploration/combined.csv'  # Замените на путь к вашему файлу
df = pd.read_csv(file_path)

# Построение графиков IoU от Score для каждого label
for label, label_name in labels.items():
    subset = df[df['label'] == label]
    plt.figure()
    plt.scatter(subset['score'], subset['iou'], s=3)
    plt.title(f'IoU vs Score for {label_name}')
    plt.xlabel('Score')
    plt.ylabel('IoU')
    plt.grid(True)
    plt.show()

# Построение общего графика IoU от Score для всех меток
plt.figure()
for label, label_name in labels.items():
    subset = df[df['label'] == label]
    plt.scatter(subset['score'], subset['iou'], s=5, label=label_name)
plt.title('IoU vs Score for All Labels')
plt.xlabel('Score')
plt.ylabel('IoU')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import random
from PIL import Image, ImageDraw
import os

In [ ]:
# Функция для рисования боксов на изображении
def draw_boxes_on_image(image_path, label_path):
    # Загрузка изображения
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)

    # Чтение файла разметки
    with open(label_path, 'r') as file:
        lines = file.readlines()

    # Рисование боксов
    for line in lines:
        parts = line.strip().split()
        x_center, y_center, width, height = map(float, parts[1:])

        # Преобразование координат
        img_width, img_height = img.size
        x_center *= img_width
        y_center *= img_height
        width *= img_width
        height *= img_height

        # Координаты боксов
        left = x_center - width / 2
        top = y_center - height / 2
        right = x_center + width / 2
        bottom = y_center + height / 2

        # Рисование бокса
        draw.rectangle([left, top, right, bottom], outline='red', width=2)

    return img

In [ ]:
# Базовый путь к изображениям
base_image_path = '/home/slava/hdd/hdd4/Datasets/lct-drones/prod_data/images'
# Базовый путь к разметкам
base_label_path = '/home/slava/hdd/hdd4/Datasets/lct-drones/prod_data/labels/'


# Фильтрация данных для IoU < 0.5 и score > 0.8
filtered_df = df[(df['iou'] > 0.4)] #& (df['score'] > 0.3)]

# Подсчет количества уникальных путей
unique_paths_count = filtered_df['filename'].nunique()

print(f"Количество уникальных путей с IoU > 0.4 и score > 0.3: {unique_paths_count}")
# Выбор 10 случайных строк
if len(filtered_df) > 10:
    filtered_sample = filtered_df.sample(n=10)
else:
    filtered_sample = filtered_df

# Отображение изображений с нарисованными боксами
for index, row in filtered_sample.iterrows():
    img_path = os.path.join(base_image_path, row['filename'])  # Объединение базового пути и имени файла
    label_filename = os.path.splitext(row['filename'])[0] + '.txt'
    label_path = os.path.join(base_label_path, label_filename)  # Формирование пути к файлу разметки

    img_with_boxes = draw_boxes_on_image(img_path, label_path)

    plt.figure()
    plt.imshow(img_with_boxes)
    plt.title(f"Filename: {row['filename']}, Score: {row['score']}, IoU: {row['iou']}")
    plt.axis('off')
    plt.show()



In [ ]:
# Извлечение уникальных путей
unique_paths = filtered_df['filename'].unique()
# Создание полного пути для каждого файла
unique_full_paths = [os.path.join(base_image_path, filename) for filename in unique_paths]

# Сохранение уникальных путей в текстовый файл
output_file = 'filtered_image_paths.txt'
with open(output_file, 'w') as file:
    for path in unique_full_paths:
        file.write(f"{path}\n")

print(f"Пути к уникальным изображениям сохранены в {output_file}")

In [ ]:
# Сейчас будем смотреть фотки с плохим и хорошим IoU

df_sorted = df.sort_values(by='iou', axis=0)

In [ ]:
df_sorted

In [ ]:
row = df_sorted.iloc[45000]
img_path = os.path.join(base_image_path, row['filename'])  # Объединение базового пути и имени файла
label_filename = os.path.splitext(row['filename'])[0] + '.txt'
label_path = os.path.join(base_label_path, label_filename)  # Формирование пути к файлу разметки

img_with_boxes = draw_boxes_on_image(img_path, label_path)
print(f"Разрешение {img_with_boxes.size}")

plt.figure()
plt.imshow(img_with_boxes)
plt.title(f"Filename: {row['filename']}, Score: {row['score']}, IoU: {row['iou']}")
plt.axis('off')
plt.show()

In [ ]:
result = get_sliced_prediction(
    Image.open(img_path),
    model,
    slice_height=1280,
    slice_width=1280,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
    verbose=True,
)
img_with_preds = get_yolo_result(result_sahi=result, category_mapping=convert_category_keys(model.category_mapping)).plot()
plt.figure()
plt.imshow(img_with_preds)
plt.axis('off')
plt.show()